In [22]:
import os
import sys
sys.path.append('..')
sys.path.append('../Automatic-Circuit-Discovery/')
sys.path.append('../tracr/')
import IPython
ipython = get_ipython()
if ipython is not None:
    ipython.magic('load_ext autoreload')
    ipython.magic('autoreload 2')
import torch as t
from torch import Tensor
import numpy as np
import matplotlib.pyplot as plt
import json

from acdc.TLACDCExperiment import TLACDCExperiment
from acdc.docstring.utils import get_all_docstring_things
from utils.prune_utils import get_3_caches, split_layers_and_heads
from ACDCPPExperiment import ACDCPPExperiment
device = t.device("cuda" if t.cuda.is_available() else "cpu")
print(device)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
cpu


/var/folders/tn/nyh0947d6sngjfvtp7hbb0jr0000gn/T/ipykernel_26576/224720274.py:9: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic('load_ext autoreload')
/var/folders/tn/nyh0947d6sngjfvtp7hbb0jr0000gn/T/ipykernel_26576/224720274.py:10: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic('autoreload 2')


In [23]:
all_docstring_items = get_all_docstring_things(num_examples=40, seq_len=5, device=device, metric_name='docstring_metric', correct_incorrect_wandb=False)

tl_model = all_docstring_items.tl_model
validation_metric = all_docstring_items.validation_metric
validation_data = all_docstring_items.validation_data
validation_labels = all_docstring_items.validation_labels
validation_patch_data = all_docstring_items.validation_patch_data
test_metrics = all_docstring_items.test_metrics
test_data = all_docstring_items.test_data
test_labels = all_docstring_items.test_labels
test_patch_data = all_docstring_items.test_patch_data

Loaded pretrained model attn-only-4l into HookedTransformer
Moving model to device:  cpu


In [24]:
def abs_docstring_metric(logits):
    return -abs(test_metrics['docstring_metric'](logits))

# ACDC++ run

In [25]:
ACDCPP_THRESHOLDS = [1,-1]
ACDC_THRESHOLDS = [1, 1.1]
RUN_NAME = 'docstring_edge_absval'


combined_exp = ACDCPPExperiment(
    tl_model,
    test_data,
    test_patch_data,
    test_metrics['docstring_metric'],
    abs_docstring_metric,
    acdc_thresholds=ACDC_THRESHOLDS,
    acdcpp_thresholds=ACDCPP_THRESHOLDS,
    run_name=RUN_NAME,
    verbose=False,
    attr_absolute_val=True,
    save_graphs_after=0,
    pruning_mode='edge',
    no_pruned_nodes_attr=1,
)
pruned_heads, num_passes, pruned_attrs = combined_exp.run()

self.current_node=TLACDCInterpNode(blocks.3.hook_resid_post, [:])


ACDC++:   0%|          | 0/2 [00:00<?, ?it/s]

acdcpp_threshold=1


acdc_threshold=1
self.current_node=TLACDCInterpNode(blocks.3.hook_resid_post, [:])


ACDC++:   0%|          | 0/2 [01:04<?, ?it/s]


KeyboardInterrupt: 

In [6]:
## Calc edge attributions
threshold_dummy = -1 # Does not make a difference when only running edge based attribution patching, as all attributions are saved in the result dict anyways
RUN_NAME = 'greaterthan_edge_absval'
acdcpp_exp = ACDCPPExperiment(tl_model,
                              test_data,
                              test_patch_data,
                              test_metrics['docstring_metric'],
                              abs_docstring_metric,
                              threshold_dummy,
                              run_name=RUN_NAME,
                              verbose=False,
                              attr_absolute_val=True,
                              save_graphs_after=0,
                              pruning_mode="edge",
                              no_pruned_nodes_attr=1
                             )
# tlacdc_exp = acdcpp_exp.setup_exp(threshold=threshold_dummy)
# acdcpp_attr = acdcpp_exp.run_acdcpp(exp=tlacdc_exp)

# # Save results
# def convert_to_torch_index(index_list):
#     return ''.join(['None' if i == ':' else i for i in index_list])

# cleaned_attrs = []
# for ((e1, i1, _), (e2, i2, _)), attr in acdcpp_attr.items():
#     cleaned_attrs.append([e1, convert_to_torch_index(str(i1)), e2, convert_to_torch_index(str(i2)), attr])
        
# with open(f'{RUN_NAME}_acdcpp_only_attrs.json', 'w') as f:
#     json.dump(cleaned_attrs, f)

TypeError: ACDCPPExperiment.__init__() missing 1 required positional argument: 'acdcpp_thresholds'

In [38]:
def eval_acdcpp(exp, acdcpp_results, threshold, attr_absolute_val=True, verbose=False):
        """
        Applying threshold to precalculated results from run_acdcpp()
        """

        for (parent, downstream_component), attr in acdcpp_results.items():
            if attr_absolute_val: 
                attr = np.abs(attr)

            # for position in exp.positions: # TODO add this back in!
            should_prune = attr < threshold
            if should_prune:
                edge_tuple = (downstream_component.hook_point_name, downstream_component.index, parent.hook_point_name, parent.index)
                exp.corr.edges[edge_tuple[0]][edge_tuple[1]][edge_tuple[2]][edge_tuple[3]].present = False
                exp.corr.remove_edge(*edge_tuple)
                if verbose:
                    print(f'Pruning {parent=} {downstream_component=}')
            else:
                if verbose: # Putting this here since tons of things get pruned when doing edges!
                    print(f'NOT PRUNING {parent=} {downstream_component=} with attribution {attr}')

        return exp

In [ ]:
# Define ACDC++ threshold
acdcpp_threshold = 0.5
acdc_threshold = 1

# Make experiment with prepruned edges according to ACDCpp experiment
full_exp = acdcpp_exp.setup_exp(threshold=acdc_threshold)
prepruned_exp = eval_acdcpp(
    exp=full_exp,
    acdcpp_results=acdcpp_attr,
    threshold=acdcpp_threshold,
)

pruned_heads, num_passes, acdcpp_attrs = acdcpp_exp.run_(prepruned_exp)

ln_final.hook_normalized
ln_final.hook_scale
blocks.3.hook_resid_post
blocks.3.hook_attn_out
blocks.3.attn.hook_result
blocks.3.attn.hook_z
blocks.3.attn.hook_pattern
blocks.3.attn.hook_attn_scores
blocks.3.attn.hook_v
blocks.3.attn.hook_k
blocks.3.attn.hook_q
blocks.3.ln1.hook_normalized
blocks.3.ln1.hook_scale
blocks.3.hook_v_input
blocks.3.hook_k_input
blocks.3.hook_q_input
blocks.3.hook_resid_pre
blocks.2.hook_resid_post
blocks.2.hook_attn_out
blocks.2.attn.hook_result
blocks.2.attn.hook_z
blocks.2.attn.hook_pattern
blocks.2.attn.hook_attn_scores
blocks.2.attn.hook_v
blocks.2.attn.hook_k
blocks.2.attn.hook_q
blocks.2.ln1.hook_normalized
blocks.2.ln1.hook_scale
blocks.2.hook_v_input
blocks.2.hook_k_input
blocks.2.hook_q_input
blocks.2.hook_resid_pre
blocks.1.hook_resid_post
blocks.1.hook_attn_out
blocks.1.attn.hook_result
blocks.1.attn.hook_z
blocks.1.attn.hook_pattern
blocks.1.attn.hook_attn_scores
blocks.1.attn.hook_v
blocks.1.attn.hook_k
blocks.1.attn.hook_q
blocks.1.ln1.hook_nor

FileNotFoundError: [Errno 2] No such file or directory: 'ims/greaterthan_edge_absval/thresh1_img_1'

In [ ]:
combined_exp = ACDCPPExperiment(tl_model,
                              test_data,
                              test_patch_data,
                              test_metrics['docstring_metric'],
                              abs_docstring_metric,
,
                              run_name=RUN_NAME,
                              verbose=False,
                              attr_absolute_val=True,
                              save_graphs_after=0,
                              pruning_mode="edge",
                              no_pruned_nodes_attr=1
                             )

In [17]:
full_exp = combined_exp.setup_exp(threshold=-1)

ln_final.hook_normalized
ln_final.hook_scale
blocks.3.hook_resid_post
blocks.3.hook_attn_out
blocks.3.attn.hook_result
blocks.3.attn.hook_z
blocks.3.attn.hook_pattern
blocks.3.attn.hook_attn_scores
blocks.3.attn.hook_v
blocks.3.attn.hook_k
blocks.3.attn.hook_q
blocks.3.ln1.hook_normalized
blocks.3.ln1.hook_scale
blocks.3.hook_v_input
blocks.3.hook_k_input
blocks.3.hook_q_input
blocks.3.hook_resid_pre
blocks.2.hook_resid_post
blocks.2.hook_attn_out
blocks.2.attn.hook_result
blocks.2.attn.hook_z
blocks.2.attn.hook_pattern
blocks.2.attn.hook_attn_scores
blocks.2.attn.hook_v
blocks.2.attn.hook_k
blocks.2.attn.hook_q
blocks.2.ln1.hook_normalized
blocks.2.ln1.hook_scale
blocks.2.hook_v_input
blocks.2.hook_k_input
blocks.2.hook_q_input
blocks.2.hook_resid_pre
blocks.1.hook_resid_post
blocks.1.hook_attn_out
blocks.1.attn.hook_result
blocks.1.attn.hook_z
blocks.1.attn.hook_pattern
blocks.1.attn.hook_attn_scores
blocks.1.attn.hook_v
blocks.1.attn.hook_k
blocks.1.attn.hook_q
blocks.1.ln1.hook_nor

self.current_node=TLACDCInterpNode(blocks.3.hook_resid_post, [:])
